In [1]:
## imports
import os
import tensorflow as tf
import numpy as np
import math
import random
import scipy.misc
#from utils import *
import pandas as pd

# Molecule data
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem 

In [7]:
## Discriminator

def discriminator(input,layers_dim, is_train, reuse=False):
    
    with tf.variable_scope('dis') as scope:
        if reuse:
            scope.reuse_variables()
                
        # Input layer (layers_dim[0]) encoding layer
        w1 = tf.get_variable('w1_dis', shape=[layers_dim[2], layers_dim[2]-3], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b1 = tf.get_variable('b1_dis', shape=[layers_dim[2]-3], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))
        dense1 = lrelu(tf.add(tf.matmul(input, w1), b1, name='dis_mat_mul_1'), 'discriminator_input_layer', leak = 0.2)
        bn1 = tf.contrib.layers.batch_norm(dense1, is_training = is_train, epsilon=1e-5, decay = 0.9,
                                           updates_collections=None, scope = 'bn1_dis')  

        # First hidedn layer (layers_dim[1]) encoding layer
        w2 = tf.get_variable('w2_dis', shape=[layers_dim[2]-3, 5], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b2 = tf.get_variable('b2_dis', shape=[5], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))
        dense2 = lrelu(tf.add(tf.matmul(bn1, w2), b2, name='dis_mat_mul_2'), 'encoder_hlayer', leak = 0.2)
        bn2 = tf.contrib.layers.batch_norm(dense2, is_training = is_train, epsilon=1e-5, decay = 0.9,
                                           updates_collections=None, scope = 'bn2_dis')


        w3 = tf.get_variable('w3_dis', shape=[5, 1], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b3 = tf.get_variable('b3_dis', shape=[1], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))

        # wgan just get rid of the sigmoid
        logits = tf.add(tf.matmul(bn2, w3), b3, name='logits')
        # dcgan
        acted_out = tf.nn.sigmoid(logits)
        return logits #, acted_out

In [8]:
## Trainer

def train():
    continue_training = False
    path = '/mnt/SSD1/data/LD50/LD50_training_data/'
    file_name = 'training_data_LD50_ht.txt'
    batch_size = 128
    fps_dim = 512
    layers_dim = np.array([256, 64, 10])
    fps = data_extract(csv_name = 'LD50_Test_data.csv')
    real = np.ones([batch_size,1])
    fake = np.zeros([batch_size,1])
    val_lr_enc = 2e-6
    val_lr_dec = 2e-6
    val_lr_dis = 2e-6
    
    with tf.variable_scope('input'):
        #real and fake image placholders
        real_fps = tf.placeholder(tf.float32, shape = [None, fps_dim], name='real_fps')
        gen_fps = tf.placeholder(tf.float32, shape=[None, fps_dim], name='gen_fps')
        dist_encode = tf.placeholder(tf.float32, shape=[None, layers_dim[2]], name='gen_fps')
        
        real_lable = tf.placeholder(tf.float32, shape=[None, 1], name='real_lable')
        fake_lable = tf.placeholder(tf.float32, shape=[None, 1], name ='fake_lable')
        
        is_train_enc = tf.placeholder(tf.bool, name='is_train_enc')
        is_train_dec = tf.placeholder(tf.bool, name= 'is_tain_dec')
        is_train_dis = tf.placeholder(tf.bool, name= 'is_train_dis')
        
        lr_enc = tf.placeholder(tf.float32, shape = [1], name='learning_rate_encoder')
        lr_dec = tf.placeholder(tf.float32, shape = [1], name='learning_rate_decoder')
        lr_dis = tf.placeholder(tf.float32, shape = [1], name='learning_rate_discriminator')
    
    # wgan
    real_encode = encoder(real_fps, fps_dim, layers_dim, is_train_enc, reuse=False)
    print(real_encode.shape)
    real_decode = decoder(real_encode, fps_dim, layers_dim, is_train_dec, reuse= False)
    
    
    real_result = discriminator(dist_encode,layers_dim, is_train_dis, reuse =False)
    fake_result = discriminator(real_encode, layers_dim, is_train_dis, reuse=True)
    
    dis_loss_real = tf.losses.mean_squared_error(real_result, real_lable)
    dis_loss_fake = tf.losses.mean_squared_error(fake_result, fake_lable)
    dis_loss = dis_loss_real + dis_loss_fake
    #dis_loss = tf.reduce_mean(fake_result) - tf.reduce_mean(real_result)  # This optimizes the discriminator.
    enc_loss = -tf.reduce_mean(fake_result)  # This optimizes the generator.
    dec_loss = tf.reduce_mean(tf.square(real_fps - real_decode))        

    t_vars = tf.trainable_variables()
    
    dis_vars = [var for var in t_vars if 'dis' in var.name]
    enc_vars = [var for var in t_vars if 'encode' in var.name]
    dec_vars = [var for var in t_vars if 'decode' in var.name]
    
    trainer_dis = tf.train.AdamOptimizer(learning_rate =lr_dis, beta1=0.9, beta2=0.999,
                                         epsilon=1e-08, use_locking=False,
                                         name='Adam_discriminator').minimize(dis_loss,var_list=dis_vars)
    trainer_dec = tf.train.AdamOptimizer(learning_rate =lr_dec, beta1=0.9, beta2=0.999,
                                         epsilon=1e-08, use_locking=False,
                                         name='Adam_decoder').minimize(dec_loss, var_list=dec_vars)
    trainer_enc = tf.train.AdamOptimizer(learning_rate =lr_enc, beta1=0.9, beta2=0.999,
                                         epsilon=1e-08, use_locking=False,
                                         name='Adam_encoder').minimize(enc_loss, var_list=enc_vars)
    
    
    # clip discriminator weights
    d_clip = [v.assign(tf.clip_by_value(v, -0.01, 0.01)) for v in dis_vars]

    
    
    fps_batch, batch_num = process_data_np(fps, batch_size = batch_size)
    
    
    total_batch = 0
    sess = tf.Session()
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    # continue training
    if continue_training:
        save_path = saver.save(sess, "/mnt/HDD1/models/LD50/model.ckpt")
        ckpt = tf.train.latest_checkpoint('/mnt/HDD1/models/LD50')
        saver.restore(sess, save_path)

    gpu_options = tf.GPUOptions(allow_growth=True)
    session_config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False,
                                    gpu_options=gpu_options)
    #threads = tf.train.start_queue_runners(sess=sess, coord=coord)


    print('batch size: %d, batch num per epoch: %d, epoch num: %d' % (batch_size, batch_num, 5000))
    print('start training...')
    for i in range(51):
        train_fps = sess.run(fps_batch)
        print("Epoch %d" % i)
        for j in range(batch_num):
            dis_iters = 5
            enc_iters = 1
            dec_iters = 1
            
            
            for k in range(dis_iters):
                
                #wgan clip weights
                sess.run(d_clip)
                train_noise = np.random.normal(loc = 0, scale = 0.3,
                                               size=[batch_size, layers_dim[2]]).astype(np.float32)
                # Update the discriminator
                _, disLoss = sess.run([trainer_dis, dis_loss],
                                    feed_dict={real_fps: train_fps[j],
                                               dist_encode: train_noise,
                                               real_lable: real,
                                               fake_lable: fake,
                                               is_train_enc: False,
                                               is_train_dis: True})

            # Update the encoder
            for k in range(enc_iters):
                # train_noise = np.random.uniform(-1.0, 1.0, size=[batch_size, random_dim]).astype(np.float32)
                _, encLoss = sess.run([trainer_enc, enc_loss],
                                    feed_dict={real_fps: train_fps[j], is_train: True})

            # print 'train:[%d/%d],d_loss:%f,g_loss:%f' % (i, j, dLoss, gLoss)
            for k in range(dec_iters):
                # train_noise = np.random.uniform(-1.0, 1.0, size=[batch_size, random_dim]).astype(np.float32)
                _, decLoss = sess.run([trainer_dec, dec_loss],
                                feed_dict={real_fps: train_fps[j], is_train: True})

        # save check point every 500 epoch
        if i%500 == 0:
            if continue_training:
                if not os.path.exists('/mnt/HDD1/models/LD50/' ):
                    os.makedirs('/mnt/HDD1/models/LD50/')
                saver.save(sess, '/mnt/HDD1/models/LD50/' + 'model' + str(i) + '.ckpt')  
        if i%5 == 0:
            if not os.path.exists(path):
                os.makedirs(path)
            if not os.path.isfile(path + file_name):
                fo = open(path +file_name, 'w+')
            else:
                fo = open(path + file_name,'a')
             
            
            fo.write('%d;%f;%f;%f\n' % (i, disLoss,  encLoss,  decLoss))
            fo.close()
            


In [9]:
## similarity test 
def sim_test():
    random_dim 
    with tf.variable_scope('input'):
        real_image = tf.placeholder(tf.float32, shape = [None, HEIGHT, WIDTH, CHANNEL], name='real_image')
        random_input = tf.placeholder(tf.float32, shape=[None, random_dim], name='rand_input')
        is_train = tf.placeholder(tf.bool, name='is_train')
    
    # wgan
    fake_image = generator(random_input, random_dim, is_train)
    real_result = discriminator(real_image, is_train)
    fake_result = discriminator(fake_image, is_train, reuse=True)
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    variables_to_restore = slim.get_variables_to_restore(include=['gen'])
    print(variables_to_restore)
    saver = tf.train.Saver(variables_to_restore)
    ckpt = tf.train.latest_checkpoint('./model/' + version)
    saver.restore(sess, ckpt)


In [10]:
## Main

if __name__ == "__main__":
    train()

(?, 10)


ValueError: Shape must be rank 0 but is rank 1 for 'Adam_discriminator/update_dis/w1_dis/ApplyAdam' (op: 'ApplyAdam') with input shapes: [10,7], [10,7], [10,7], [], [], [1], [], [], [], [10,7].